In [85]:
import datetime
import hyperopt
import itertools
import numpy
import operator
import pandas
import random
import requests
from IPython.display import Audio
from sklearn import (
    ensemble,
    linear_model,
    metrics,
    model_selection,
    naive_bayes,
    neighbors,
    neural_network,
    svm,
)
import time
import tqdm
import xgboost

### Read and Filter

In [111]:
dtype = {'smoke': float, 'alco': float, 'active': float}

def fix_ap(value):
    value = abs(value)
    while value > 500.0:
        value /= 10.0
    return value

def read_csv(filename):
    frame = pandas.read_csv(filename, sep=';', header=0, na_values='None', dtype=dtype).drop(['id'], axis=1)
    frame['ap_hi'] = frame['ap_hi'].apply(fix_ap)
    frame = pandas.get_dummies(frame, columns=[
        'gender',
        'cholesterol',
        'gluc',
        'smoke',
        'active',
        'alco',
    ])
    frame = frame.assign(
        # aged_smoke_0=(frame['smoke_0.0'] * frame['age']),
        # aged_smoke_1=(frame['smoke_1.0'] * frame['age']),
        # aged_active_0=(frame['active_0.0'] * frame['age']),
        # aged_active_1=(frame['active_1.0'] * frame['age']),
        # aged_alco_0=(frame['alco_0.0'] * frame['age']),
        # aged_alco_1=(frame['alco_1.0'] * frame['age']),
        bmi=(frame['weight'] / frame['height'] ** 2),
        # smoke_1_active_0=(frame['smoke_1.0'] * frame['active_0.0']),
        # aged_cholesterol_1=(frame['cholesterol_1'] * frame['age']),
        # aged_cholesterol_2=(frame['cholesterol_2'] * frame['age']),
        # aged_cholesterol_3=(frame['cholesterol_3'] * frame['age']),
        # gender_1_smoke_1=(frame['gender_1'] * frame['smoke_1.0']),
        # gender_2_smoke_1=(frame['gender_2'] * frame['smoke_1.0']),
    )
    return frame

In [112]:
train = read_csv('train.csv')

X = train.drop('cardio', axis=1).values
y = train['cardio'].values
print(f'X: {X.shape}')
print(f'y: {y.shape}')

X: (70000, 20)
y: (70000,)


In [113]:
test = read_csv('test.csv')

X_test = test.values
print(f'X_test: {X_test.shape}')

X_test: (30000, 20)


In [114]:
train.head()

,age,height,weight,ap_hi,ap_lo,cardio,gender_1,gender_2,cholesterol_1,cholesterol_2,...,gluc_1,gluc_2,gluc_3,smoke_0.0,smoke_1.0,active_0.0,active_1.0,alco_0.0,alco_1.0,bmi
0,18393,168,62.0,110.0,80,0,0,1,1,0,...,1,0,0,1,0,0,1,1,0,0.002197
1,20228,156,85.0,140.0,90,1,1,0,0,0,...,1,0,0,1,0,0,1,1,0,0.003493
2,18857,165,64.0,130.0,70,1,1,0,0,0,...,1,0,0,1,0,1,0,1,0,0.002351
3,17623,169,82.0,150.0,100,1,0,1,1,0,...,1,0,0,1,0,0,1,1,0,0.002871
4,17474,156,56.0,100.0,60,0,1,0,1,0,...,1,0,0,1,0,1,0,1,0,0.002301


In [115]:
test.head()

,age,height,weight,ap_hi,ap_lo,gender_1,gender_2,cholesterol_1,cholesterol_2,cholesterol_3,gluc_1,gluc_2,gluc_3,smoke_0.0,smoke_1.0,active_0.0,active_1.0,alco_0.0,alco_1.0,bmi
0,18888,154,85.0,130.0,80,1,0,1,0,0,1,0,0,1,0,0,1,1,0,0.003584
1,19042,170,69.0,130.0,90,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0.002388
2,20432,160,70.0,120.0,75,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0.002734
3,18133,185,94.0,130.0,90,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0.002747
4,16093,170,76.0,120.0,80,0,1,1,0,0,1,0,0,1,0,0,1,1,0,0.002630


### Predict and Save

In [65]:
def fit_predict_save(estimator, filename):
    estimator.fit(X, y)
    for name, importance in sorted(zip(test, estimator.feature_importances_), key=operator.itemgetter(1), reverse=True):
        print(f'{name}: {importance:.7f}')
    numpy.savetxt(filename, estimator.predict_proba(X_test)[:, 1], fmt='%f')

### XGBoost

In [109]:
def search_xgb(params, **param_grid):
    start_time = time.time()
    grid_search = model_selection.GridSearchCV(
        xgboost.XGBClassifier(nthread=5, seed=0, **params),
        param_grid,
        scoring='neg_log_loss',
        refit=False,
        cv=model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=0),
    ).fit(X, y)
    print(f'[{(time.time() - start_time) / 60.0:.2f} min] {grid_search.best_score_:.7f} {grid_search.best_params_} | {param_grid}')
    return grid_search

In [79]:
'''
xgb_params = {
    'n_estimators': 5, 
    'reg_lambda': 1.0,
    'scale_pos_weight': 1.0,
    'base_score': 0.5,
}
'''

In [116]:
def make_int_grid(params, param_name, delta):
    n = params.get(param_name, 1)
    return {param_name: list(range(max(1, n - delta), n + delta + 1))}

def make_float_grid(params, param_name, delta, min_, max_):
    x = params.get(param_name, 0.0)
    return {param_name: [max(min_, x - delta), x, min(max_, x + delta)]}

for _ in itertools.count(start=1):
    last_params = dict(xgb_params)
    xgb_params.update(search_xgb(xgb_params, **make_float_grid(xgb_params, 'base_score', 0.01, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_float_grid(xgb_params, 'scale_pos_weight', 0.02, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_float_grid(xgb_params, 'reg_lambda', 0.01, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_float_grid(xgb_params, 'reg_alpha', 0.01, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_float_grid(xgb_params, 'gamma', 0.01, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_int_grid(xgb_params, 'max_depth', 1)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_int_grid(xgb_params, 'n_estimators', 3)).best_params_)
    if xgb_params == last_params:
        break

[0.34 min] -0.5390045 {'base_score': 0.51} | {'base_score': [0.5, 0.51, 0.52]}
[0.36 min] -0.5390045 {'scale_pos_weight': 0.99} | {'scale_pos_weight': [0.97, 0.99, 1.0]}
[0.37 min] -0.5390045 {'reg_lambda': 1.0} | {'reg_lambda': [0.99, 1.0, 1.0]}
[0.36 min] -0.5390045 {'reg_alpha': 0.02} | {'reg_alpha': [0.01, 0.02, 0.03]}
[0.35 min] -0.5389759 {'gamma': 0.16999999999999998} | {'gamma': [0.16999999999999998, 0.18, 0.19]}
[0.34 min] -0.5389759 {'max_depth': 5} | {'max_depth': range(4, 7)}
[0.81 min] -0.5389418 {'n_estimators': 91} | {'n_estimators': range(89, 96)}
[0.35 min] -0.5389418 {'base_score': 0.51} | {'base_score': [0.5, 0.51, 0.52]}
[0.34 min] -0.5389418 {'scale_pos_weight': 0.99} | {'scale_pos_weight': [0.97, 0.99, 1.0]}
[0.35 min] -0.5389418 {'reg_lambda': 1.0} | {'reg_lambda': [0.99, 1.0, 1.0]}
[0.35 min] -0.5389418 {'reg_alpha': 0.02} | {'reg_alpha': [0.01, 0.02, 0.03]}
[0.35 min] -0.5389418 {'gamma': 0.16999999999999998} | {'gamma': [0.15999999999999998, 0.1699999999999999

In [117]:
fit_predict_save(xgboost.XGBClassifier(nthread=5, **xgb_params), f'xgboost.txt')

age: 0.2569386
bmi: 0.1698907
ap_hi: 0.1169050
height: 0.0988225
weight: 0.0975610
ap_lo: 0.0748528
cholesterol_1: 0.0315391
active_0.0: 0.0269134
cholesterol_3: 0.0239697
gluc_3: 0.0218671
gluc_1: 0.0155593
alco_0.0: 0.0147183
smoke_0.0: 0.0142977
gluc_2: 0.0130362
cholesterol_2: 0.0117746
gender_1: 0.0113541
gender_2: 0.0000000
smoke_1.0: 0.0000000
active_1.0: 0.0000000
alco_1.0: 0.0000000
